In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from pathlib import Path
import pymc as pm
import arviz as az
import multiprocessing as mp
from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp
from estival.wrappers import nevergrad as eng
from estival.utils.parallel import map_parallel
import nevergrad as ng

from tbdynamics.constants import BURN_IN, OPTI_DRAWS
from autumn.infrastructure.remote import springboard
from tbdynamics.calib_utils import get_bcm
import pandas as pd
from estival.utils.sample import SampleTypes

In [ ]:
params = {
    "start_population_size": 2300000.0,
    "seed_time": 1810.0,
    "seed_num": 10.0,
    "seed_duration": 3.0,
    "screening_scaleup_shape": 0.1,
    "screening_inflection_time": 1993.0,
    "screening_end_asymp": 0.65,
}

bcm = get_bcm(params)

In [ ]:
def optimize_ng(sample):
    opt = eng.optimize_model(bcm, budget=2000, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=4)
    rec= opt.minimize(2000)
    return rec.value[1]

In [ ]:
opt_samples = map_parallel(optimize_ng, bcm.sample.lhs(16, SampleTypes.LIST_OF_DICTS), n_workers=4, mode="process")

In [ ]:
lle_samps = esamp.likelihood_extras_for_samples(opt_samples, bcm)

In [ ]:
best_ll_idx = lle_samps.sort_values("logposterior",ascending=False).index

In [ ]:
init_samps= bcm.sample.convert(opt_samples,SampleTypes.SAMPLEITERATOR)[best_ll_idx].convert(SampleTypes.LIST_OF_DICTS)[0]

In [ ]:
init_samps

In [ ]:
with pm.Model() as model:
    variables = epm.use_model(bcm)
    idata_raw = pm.sample(
        step=[pm.DEMetropolisZ(variables)],
        draws=15000,
        tune=3000,
        cores=4,
        discard_tuned_samples=False,
        chains=4,
        progressbar=True,
        initvals=init_samps,
    )

In [ ]:
import arviz as az

In [ ]:
az.summary(idata_raw)

In [ ]:
az.plot_trace(idata_raw, figsize=(16,3.2*len(idata_raw.posterior)) ,compact=False)